# Le Deep Learning

Le principe est le même que celui expliqué précédemment chaque neurone execute la fonction f(x)= W * x + b

La seule différence réside dans le fait que x, W et b sont des  matrices (tensor) contenant les output des neuronnes précédents et les valeurs corrigées.

Chaque neuronne est activé (on converti le score en probabilité). Cette probabilité est ensuite transmise aux neuronnes suivants. On répète le process pour chaque couche.

C'est ce qu'on appelle le *Forward Propagation*

Lorsque l'on arrive au dernier layer on possède donc un Delta / Erreur qui rend compte de l'écart entre la prédiction et la valeur recherchée.

Ainsi on va donc corriger chaque matrices W et b de chaque neuronne pour arriver au résultat désiré.

Ce n'est qu'une réccursive de ce que nous avons vu précédemment.

On appelle cela *Back Propagation*.




# Introduire la regression

In [1]:
%matplotlib inline               
import pandas as pd              # A beautiful library to help us work with data as tables
import numpy as np               # So we can use number matrices. Both pandas and TensorFlow need it. 
import matplotlib.pyplot as plt  # Visualize the things
import tensorflow as tf          # Fire from the gods
import __future__
import random
from datetime import datetime

print("everything is imported")
now = datetime.utcnow().strftime("%y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run".format(root_logdir, now)

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


everything is imported


In [2]:
#Comme d'habitude on charge le jeu de données

df = pd.read_csv("housing.csv")
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [3]:
#On change les valeurs catégoriques en valeur numériques
df['ocean_proximity'] = df['ocean_proximity'].map({'NEAR BAY':1 , 'NEAR OCEAN':0, '<1H OCEAN': 2, "INLAND":3})
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,1.0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,1.0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,1.0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,1.0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,1.0


In [4]:
#On normalise le jeu de données
df_numerical = df.drop(["ocean_proximity"], axis=1)
df_norm = (df_numerical - df_numerical.mean()) / (df_numerical.max() - df_numerical.min())
df_norm["ocean_proximity"] = df["ocean_proximity"]
df_norm.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-0.264970,0.238910,0.242363,-0.044655,-0.063450,-0.030928,-0.061427,0.307205,0.506687,1.0
1,-0.263974,0.236784,-0.149794,0.113516,0.088164,0.027342,0.104993,0.305563,0.312667,1.0
2,-0.265966,0.235721,0.458049,-0.029726,-0.053984,-0.026051,-0.053041,0.233564,0.299471,1.0
3,-0.266962,0.235721,0.458049,-0.034635,-0.047000,-0.024313,-0.046134,0.122235,0.277203,1.0
4,-0.266962,0.235721,0.458049,-0.025657,-0.040017,-0.024117,-0.039556,-0.001688,0.279059,1.0


In [5]:
#Définissons nos données d'entrées et de sorties
inputX = df_norm.loc[:, ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'population', 'households', 'median_income', 'ocean_proximity']].as_matrix() #On converti en matrices pour Tensorflow
inputY = df_norm.loc[:, ["median_house_value"]].as_matrix()
inputX

array([[-0.26496968,  0.23890952,  0.24236301, ..., -0.06142734,
         0.30720466,  1.        ],
       [-0.26397366,  0.23678412, -0.14979385, ...,  0.10499265,
         0.3055633 ,  1.        ],
       [-0.26596569,  0.23572142,  0.45804929, ..., -0.05304057,
         0.2335643 ,  1.        ],
       ...,
       [-0.16437207,  0.4036279 , -0.22822522, ..., -0.01094223,
        -0.14969938,  3.        ],
       [-0.17433223,  0.4036279 , -0.20861738, ..., -0.02475574,
        -0.13816851,  3.        ],
       [-0.1663641 ,  0.39725171, -0.24783307, ...,  0.0050091 ,
        -0.10221038,  3.        ]])

In [7]:
print(inputY[1])

[0.31266713]


# Avec un réseau multicouche

In [8]:
learning_rate = 0.0001
training_epochs = 2000
display_step = 100
n_samples = inputY.size
with tf.device('/device:GPU:0'):

    X = tf.placeholder(tf.float32, [1, inputX.shape[1]])
    Y = tf.placeholder(tf.float32, [1, 1])

    n_neurons_1 = 128
    n_neurons_2 = 64
    n_neurons_3 = 32

    #couche 1
    w_hidden_1 = tf.Variable(tf.ones([8, n_neurons_1])) #9 vecteurs (un pour chaque entrée de la taille du nombre de neuronnes de la couche suivante)
    b_hidden_1 = tf.Variable(tf.ones([n_neurons_1])) #1 vecteur de la taille de la couche suivante, 1 biais par liaison
    #couche2
    w_hidden_2 = tf.Variable(tf.ones([n_neurons_1, n_neurons_2]))
    b_hidden_2 = tf.Variable(tf.ones([n_neurons_2]))

    w_hidden_3 = tf.Variable(tf.ones([n_neurons_2, n_neurons_3]))
    b_hidden_3 = tf.Variable(tf.ones([n_neurons_3]))


    #sortie
    w_out = tf.Variable(tf.ones([n_neurons_3, 1]))
    b_out = tf.Variable(tf.ones([1,1]))
    #Graph
    #Maintenant il faut lier chaque couche avec la fonction W*x+b sans oublier la fonction d'activation, qui ici sera un relu

    layer_1 = tf.nn.relu(tf.add(tf.matmul(X, w_hidden_1), b_hidden_1))
    layer_2 = tf.nn.relu(tf.add(tf.matmul(layer_1, w_hidden_2), b_hidden_2))
    layer_3 = tf.nn.relu(tf.add(tf.matmul(layer_2, w_hidden_3), b_hidden_3))
    prediction = tf.add(tf.matmul(layer_3, w_out), b_out)
    accuracy = 1/tf.divide(tf.subtract(Y[0][0], prediction[0][0]),Y[0][0])
    #définition des fonctions de cout et optimisation
    cost= tf.reduce_mean(tf.squared_difference(Y, prediction))
    opt= tf.train.AdamOptimizer(learning_rate).minimize(cost)
    #initialisation
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [12]:
#pb cost nan
for i in range(training_epochs):
    average_cost = 0
    average_accuracy = 0
    random_index = random.randint(1, len(inputX)-110)
    batchX = inputX[random_index:random_index+100]
    batchY = inputY[random_index:random_index+100]
    #print(batchX)
    for a in batchX:
        for b in a:
            if(np.isnan(b)):
                print(type(b))
                print(b)
    for j in range(100):
        x = batchX[j]
        y = batchY[j]
        
        #print(sess.run(prediction, feed_dict={X:[x]}))
        #print(sess.run([layer_1, layer_2, layer_3, layer_4], feed_dict={X:[x]}))
        _,cc, local_accuracy = sess.run([opt,cost, accuracy], feed_dict={X:[x], Y:[y]})
        #print(sess.run([Y, prediction], feed_dict={X:[x], Y: [y]}))
        #print(cc)
        average_cost += cc
        average_accuracy += local_accuracy / 100
        test_result = sess.run(prediction, feed_dict={X:[inputX[264]]})[0]
        price_pred = test_result * (df_numerical["median_house_value"].max() - df_numerical["median_house_value"].min()) + df_numerical["median_house_value"].mean()
        real_price = inputY[264][0] * (df_numerical["median_house_value"].max() - df_numerical["median_house_value"].min()) + df_numerical["median_house_value"].mean()
    if (i) % display_step == 0:
        print("Training step:", '%04d' % (i), "cost=", "{:.9f}".format(cc), "Y=", real_price, "Predicted=", price_pred)

Training step: 0000 cost= 616945221632.000000000 Y= 102800.0 Predicted= [2.592129e+11]
Training step: 0100 cost= 5323682816.000000000 Y= 102800.0 Predicted= [1.971897e+10]
Training step: 0200 cost= 13160666.000000000 Y= 102800.0 Predicted= [1.2844684e+09]


KeyboardInterrupt: 